<a href="https://colab.research.google.com/github/nicokier/KeyBERT/blob/master/Compartir_Search_Intent_de_Keys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Script generado por Álvaro Peña @isocialwebseo en twitter. Tienes libertad para modificar, mejorar y compartir parte o la totalidad del contenido de este script. 
# Eso sí, haz un uso responsable del mismo. 

#Instalamos las dependencias necesarias
!pip install openai pandas requests google-colab

In [ ]:
# Subimos el archivo CSV con nombre de archivo: keywords.csv que contiene las keywords extraidas de Ahrefs, semrush, GSC... Tienen que estar en una única columna y la primera casillas llamarse keywords
from google.colab import files
uploaded = files.upload()

import openai
import re
import base64
import requests
import json
import pandas as pd
from google.colab import data_table

# Leemos el archivo CSV que subimos al entorno de Colab, tienen que estar todas las keys en una columna con la cabecera "keywords"
df = pd.read_csv('keywords.csv')

# Convertimos el contenido del DataFrame a una cadena de texto en formato CSV
csv_string = df.to_csv(index=False)

# Convertimos la cadena de texto en formato CSV a una cadena de texto donde cada fila del CSV es una línea
csv_lines = csv_string.split('\n')
text_lines = []
for line in csv_lines:
    elements = line.split(',')
    text_lines.append(','.join(elements))

# Eliminamos la primera línea de título de la lista
text_lines = text_lines[1:]

# Volvemos a unir las líneas en una única cadena de texto
text = '\n'.join(text_lines)

#RELLENAR: Añade aquí tu API de OpenAI
openai.api_key = "TU-API-KEY-OPENAI"

# -------------------------

# Crea un contador de keywords procesadas y un índice de lote inicial
contador_keywords = 0
indice_lote = 0

# Calcular el numero total de keywords
num_keywords = len(df['keywords'])

# Crea las listas vacías para cada columna de la tabla
keywords = []
intencion = []
etapa = []


# Mientras el contador sea menor al número total de keywords en el archivo CSV
while contador_keywords < num_keywords:
  # Obtener el lote de 100 keywords
  lote = df['keywords'][indice_lote:indice_lote+100]
  
  # Convertimos el lote de keywords a una cadena de texto
  lote_string = '\n'.join(lote)
  
  prompt = f"Para el siguiente listado de palabras: \n\n {lote_string} \n \n devuelve por cada una de las keywords la intención de búsqueda (Informacional, Transaccional o Navegacional) y la etapa del embudo de conversión (Descubrimiento, Consideración o Conversión). Habrá una palabra por línea con el formato: Keyword | Intención | Etapa"

  # Llamamos a OpenAI para que atribuya las intenciones y etapas de las keys
  categorizacion = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      max_tokens=1500,
      temperature=0.7,
  )

  # Procesar las keywords categorizadas
  keywords_categorizadas = categorizacion.choices[0].text.split('\n')

  # Itera sobre cada elemento de la lista
  for elemento in keywords_categorizadas:
    # Divide la cadena en subcadenas utilizando el delimitador '|'
    subcadenas = elemento.split('|')
    # Si hay tres subcadenas, asigna cada subcadena a una variable temporal
    if len(subcadenas) == 3:
        keyword, intencion_temp, etapa_temp = subcadenas
    # Si no hay tres subcadenas, asigna las subcadenas disponibles a las variables temporales
    # y deja las variables restantes vacías
    else:
        keyword = subcadenas[0] if len(subcadenas) > 0 else ''
        intencion_temp = subcadenas[1] if len(subcadenas) > 1 else ''
        etapa_temp = subcadenas[2] if len(subcadenas) > 2 else ''
    # Añade cada subcadena a la lista correspondiente
    keywords.append(keyword)
    intencion.append(intencion_temp)
    etapa.append(etapa_temp)

  # Aumenta el contador de keywords procesadas y el índice de lote
  contador_keywords += len(lote)
  indice_lote += 100

# Crea un DataFrame de Pandas a partir de las tres listas
tabla = pd.DataFrame({'Keywords': keywords, 'Intencion': intencion, 'Etapa': etapa})

# Muestra la tabla
display(tabla)

Saving keywords.csv to keywords (7).csv


,Keywords,Intencion,Etapa
0,,,
1,,,
2,creatina,Informacional,Descubrimiento
3,creatina growth,Informacional,Descubrimiento
4,narguile,Informacional,Descubrimiento
...,...,...,...
501,Mesa de sinuca,Transaccional,Conversión
502,Fita de led,Transaccional,Conversión
503,Poco x3,Informacional,Descubrimiento
504,Celta 2015,Informacional,Descubrimiento
